In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 결측치 대체 알고리즘

## 0) DataFrame.fillna() 활용
* DataFrame.fillna(value=None, method=None, axis=None, inplace=False, limit=None, downcast=None)
    * parameter
        * value : scalar, dict, Series, or DataFrame
        * method : {'backfill', 'bfill', 'pad', 'ffill', None}, default None

## 1) SimpleImputer
* 한 특성의 통계 값을 사용하여 결측치를 대체
* 주요 parameter
    * missing_values : int, float, str, np.nan, None or pandas.NA, default=np.nan
    * strategy : str {'mean', 'medain', 'most_frequent', 'constant'} default=’mean’
    * fill_value : str or numerical value, default=None
        * strategy = 'constant'인 경우 지정
    * copybool, default=True

In [2]:
from sklearn.impute import SimpleImputer

df=pd.DataFrame(np.random.randn(10,8),columns=list('01234567'))

indices=[[1,2],[1,6],[1,7],[4,5],[4,6],[5,6],[7,2],[8,4],[9,4]]

for index in indices:
    df.iloc[index[0],index[1]]=np.nan

df

,0,1,2,3,4,5,6,7
0,0.051877,0.177566,-1.908825,-0.197848,-0.712544,-0.465912,0.928295,0.353415
1,0.508283,0.392080,NaN,1.284454,-0.228970,0.931980,NaN,NaN
2,0.578315,0.521501,-0.536710,-0.290153,-1.810798,-0.091317,-1.439049,0.317503
3,0.434185,0.788339,0.456843,0.306642,1.008544,-0.891753,0.359710,1.142842
4,0.492746,-0.594298,-0.256867,0.937637,1.488651,NaN,NaN,-2.535786
5,0.408680,0.313152,1.382635,-1.556503,1.525769,-2.293575,NaN,-1.315476
6,0.847734,-0.364045,0.723377,-0.274852,-0.429604,0.008002,0.144875,0.929042
7,-0.188939,-0.275927,NaN,-0.656610,1.020099,1.286130,0.450713,1.417044
8,1.251551,0.942231,-0.718525,0.402444,NaN,-0.443837,-0.817411,1.764483
9,-2.246375,-1.425264,0.735784,-1.058062,NaN,0.643453,0.055378,-0.350576


In [3]:
df_tmp=df
imputer = SimpleImputer(strategy='mean')
df_simple=imputer.fit_transform(df_tmp)
df_simple=pd.DataFrame(df_simple,columns=df.columns)
df_simple

,0,1,2,3,4,5,6,7
0,0.051877,0.177566,-1.908825,-0.197848,-0.712544,-0.465912,0.928295,0.353415
1,0.508283,0.392080,-0.015286,1.284454,-0.228970,0.931980,-0.045356,0.191388
2,0.578315,0.521501,-0.536710,-0.290153,-1.810798,-0.091317,-1.439049,0.317503
3,0.434185,0.788339,0.456843,0.306642,1.008544,-0.891753,0.359710,1.142842
4,0.492746,-0.594298,-0.256867,0.937637,1.488651,-0.146314,-0.045356,-2.535786
5,0.408680,0.313152,1.382635,-1.556503,1.525769,-2.293575,-0.045356,-1.315476
6,0.847734,-0.364045,0.723377,-0.274852,-0.429604,0.008002,0.144875,0.929042
7,-0.188939,-0.275927,-0.015286,-0.656610,1.020099,1.286130,0.450713,1.417044
8,1.251551,0.942231,-0.718525,0.402444,0.232643,-0.443837,-0.817411,1.764483
9,-2.246375,-1.425264,0.735784,-1.058062,0.232643,0.643453,0.055378,-0.350576


## 2) IterativeImputer
* 다른 특성을 통해 예측하여 결측치를 대체
* ```IterativeImputer(estimator=None, *, missing_values=nan, sample_posterior=False, max_iter=10, tol=0.001, n_nearest_features=None, initial_strategy='mean', imputation_order='ascending', skip_complete=False, min_value=-inf, max_value=inf, verbose=0, random_state=None, add_indicator=False)```
* parameter 많음...^^
* **IterativeImputer 클래스는 아직 실험적이기 때문에 import enable_iterative_imputer 필요**

In [4]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp_mean = IterativeImputer(random_state=0)
imp_mean.fit([[7, 2, 3], [4, np.nan, 6], [10, 5, 9]])

X = [[np.nan, 2, 3], [4, np.nan, 6], [10, np.nan, 9]]
imp_mean.transform(X)

array([[ 6.95847623,  2.        ,  3.        ],
       [ 4.        ,  2.6000004 ,  6.        ],
       [10.        ,  4.99999933,  9.        ]])

## 3) MICE(Multiple Imputation by Chained Equation)

<img src="./img/mice.PNG" width="700" height="500">

* MICE는 연쇄 등식을 이용한 다중대치로, 과정은 아래와 같다
    * 1. 결측치를 다른 모든 변수를 사용하여 예측
    * 2. 모든 결측치를 채운 데이터 셋을 m(=3)개에 대해 with()를 사용하여 통계모형을 적용
    * 3. pool()을 사용하여 분석결과를 하나로 통합

In [7]:
# !pip install impyute

In [8]:
df

,0,1,2,3,4,5,6,7
0,0.051877,0.177566,-1.908825,-0.197848,-0.712544,-0.465912,0.928295,0.353415
1,0.508283,0.392080,NaN,1.284454,-0.228970,0.931980,NaN,NaN
2,0.578315,0.521501,-0.536710,-0.290153,-1.810798,-0.091317,-1.439049,0.317503
3,0.434185,0.788339,0.456843,0.306642,1.008544,-0.891753,0.359710,1.142842
4,0.492746,-0.594298,-0.256867,0.937637,1.488651,NaN,NaN,-2.535786
5,0.408680,0.313152,1.382635,-1.556503,1.525769,-2.293575,NaN,-1.315476
6,0.847734,-0.364045,0.723377,-0.274852,-0.429604,0.008002,0.144875,0.929042
7,-0.188939,-0.275927,NaN,-0.656610,1.020099,1.286130,0.450713,1.417044
8,1.251551,0.942231,-0.718525,0.402444,NaN,-0.443837,-0.817411,1.764483
9,-2.246375,-1.425264,0.735784,-1.058062,NaN,0.643453,0.055378,-0.350576


In [9]:
from impyute.imputation.cs import mice

df_tmp=df
df_mice=mice(df_tmp.values)
df_mice=pd.DataFrame(df_mice)

df_mice

,0,1,2,3,4,5,6,7
0,0.051877,0.177566,-1.908825,-0.197848,-0.712544,-0.465912,0.928295,0.353415
1,0.508283,0.392080,-0.015286,1.284454,-0.228970,0.931980,-8.712757,2.612122
2,0.578315,0.521501,-0.536710,-0.290153,-1.810798,-0.091317,-1.439049,0.317503
3,0.434185,0.788339,0.456843,0.306642,1.008544,-0.891753,0.359710,1.142842
4,0.492746,-0.594298,-0.256867,0.937637,1.488651,-0.146314,12.215642,-2.535786
5,0.408680,0.313152,1.382635,-1.556503,1.525769,-2.293575,12.746780,-1.315476
6,0.847734,-0.364045,0.723377,-0.274852,-0.429604,0.008002,0.144875,0.929042
7,-0.188939,-0.275927,-0.015286,-0.656610,1.020099,1.286130,0.450713,1.417044
8,1.251551,0.942231,-0.718525,0.402444,0.232643,-0.443837,-0.817411,1.764483
9,-2.246375,-1.425264,0.735784,-1.058062,0.232643,0.643453,0.055378,-0.350576


## 4) KNNImputer
* KNN을 활용하여 결측치를 처리하는 알고리즘
* parameters
    * missing_values : int, float, str, np.nan or None, default=np.nan
    * n_neighbors : int, default=5
    * weights : {‘uniform’, ‘distance’} or callable, default=’uniform’
    * metric : {‘nan_euclidean’} or callable, default=’nan_euclidean’
    * copy : bool, default=True

In [10]:
from sklearn.impute import KNNImputer

In [11]:
df=pd.DataFrame(np.random.randn(10,10),columns=list('0123456789'))

indices=[[1,2],[1,6],[1,7],[4,6],[5,6],[8,4],[7,2],[9,9]]

for index in indices:
    df.iloc[index[0],index[1]]=np.nan

df

,0,1,2,3,4,5,6,7,8,9
0,-0.554429,-1.332588,-0.515174,0.322426,-0.350170,1.525940,0.383762,-0.194480,0.676047,-0.206176
1,1.683518,-0.120954,NaN,-1.516430,0.423903,1.456328,NaN,NaN,-0.175874,-1.064695
2,1.334687,-0.364129,-1.755069,0.014619,1.087764,1.248069,-1.263989,-0.746976,0.520227,0.450508
3,0.478051,0.021860,-0.468999,-0.303238,-2.316239,-0.943462,-1.563846,-1.180564,1.644476,-0.115922
4,1.804811,-0.154562,0.609110,-0.197737,-0.182005,-0.123589,NaN,-3.371816,-0.020058,0.951314
5,-0.881251,-0.979375,-0.739429,-0.339337,0.702966,-0.881325,NaN,-0.536709,-0.278573,0.592155
6,0.634576,0.552915,-0.202280,0.863491,1.102989,-0.132172,-1.444918,-0.210730,0.380222,-0.191747
7,-0.009057,-0.694542,NaN,-0.323285,0.566963,0.662263,-0.728739,1.082072,-1.017473,1.378802
8,1.474123,-1.281768,0.284401,-0.058296,NaN,0.958816,-1.391558,-0.120260,-1.004555,-0.963299
9,1.524586,-0.027971,-1.695546,-0.484576,-0.199943,2.559832,-1.686763,-1.166181,0.692104,NaN


In [12]:
df_tmp=df
imputer = KNNImputer(n_neighbors=4)
df_knn=imputer.fit_transform(df_tmp)
df_knn=pd.DataFrame(df_knn,columns=df.columns)
df_knn

,0,1,2,3,4,5,6,7,8,9
0,-0.554429,-1.332588,-0.515174,0.322426,-0.350170,1.525940,0.383762,-0.194480,0.676047,-0.206176
1,1.683518,-0.120954,-0.639276,-1.516430,0.423903,1.456328,-1.446807,-1.351308,-0.175874,-1.064695
2,1.334687,-0.364129,-1.755069,0.014619,1.087764,1.248069,-1.263989,-0.746976,0.520227,0.450508
3,0.478051,0.021860,-0.468999,-0.303238,-2.316239,-0.943462,-1.563846,-1.180564,1.644476,-0.115922
4,1.804811,-0.154562,0.609110,-0.197737,-0.182005,-0.123589,-1.416078,-3.371816,-0.020058,0.951314
5,-0.881251,-0.979375,-0.739429,-0.339337,0.702966,-0.881325,-0.763471,-0.536709,-0.278573,0.592155
6,0.634576,0.552915,-0.202280,0.863491,1.102989,-0.132172,-1.444918,-0.210730,0.380222,-0.191747
7,-0.009057,-0.694542,-0.802988,-0.323285,0.566963,0.662263,-0.728739,1.082072,-1.017473,1.378802
8,1.474123,-1.281768,0.284401,-0.058296,0.795405,0.958816,-1.391558,-0.120260,-1.004555,-0.963299
9,1.524586,-0.027971,-1.695546,-0.484576,-0.199943,2.559832,-1.686763,-1.166181,0.692104,-0.445915


In [13]:
# !pip install missingpy

In [ ]:
# from missingpy import KNNImputer
# imputer = KNNImputer()
# df_missforest = imputer.fit_transform(X)
# df_missforest=pd.DataFrame(df_missforest)
# df_missforest

## [5] MissForest
* Random Forest를 활용하여 결측치를 처리하는 알고리즘에서 
* KNN Imputer보다 성능이 우수함 

In [ ]:
# # !pip install missingpy

In [14]:
# from missingpy import MissForest

# imputer = MissForest()
# df_missforest = imputer.fit_transform(X)
# df_missforest=pd.DataFrame(df_missforest)
# df_missforest

# 2. 다양한 회귀모델

 * Linear Regression, KNeighborsRegressor, DecisionTreeRegressor, RandomForestRegressor 외
 * 선형회귀 모델인 Lasso, Ridge, ElasticNet 
 * Boosting 기법을 활용한 모델인 AdaBoost / GBM(Gradient Boosting Machine) / XGBoost / LightGBM / CatBoost


In [15]:
# !pip install xgboost
# !pip install lightgbm
# !pip install catboost

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [8]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import * 

boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['target']=boston.target

df.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88,11.9


In [9]:
x=df.drop('target', axis=1)
y=df['target']

x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.3)

scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

x_train_s = pd.DataFrame(x_train_s, columns = list(x_train))
x_val_s = pd.DataFrame(x_val_s, columns = list(x_val))

## [0] 선형 회귀 분석의 4가지 가정
* 선형 회귀 분석은 아래 4가지 가정이 충족되어야 적절하다고 판단할 수 있다.

* 1) 선형성
    * 종속변수와 독립변수 간의 선형관계

* 2) 독립성
    * 독립 변수들 간의 통계적 독립성( 비 다중공선성 ) 

*  3) 등분산성
    * 잔차들의 분산이 일정

* 4) 정규성
    * 잔차들의 분포가 정규분포를 이룸

## [0] OLS(Ordinary Least Square)
* 최소 제곱 선형 회구 모델 구현
* 비용함수로는 제곱 오차합(SSE) 사용

<img src="./img/sse.PNG" width="200" height="150">

## [1] 선형 회귀 모델
* Lasso와 Ridge는 Linear Regression의 단점을 보완환 모델   
* 규제 : 부가 정보를 손실에 더해 과대적합 문제를 방지하는 방법으로, 복잡도에 대한 패널티를 유도하여 모델 파라미터의 값을 감소시킴.
* 규제 강도는 lambda에 해당하는 alpha 매개변수로 조절

### 1) Ridge Regression
* 릿지 회귀는 최소 제곱 비용함수에 가중치의 제곱합을 추가한 L2 규제 모델
* ```ridge = Ridge(alpha=1.0)```
<img src="./img/ridge.PNG" width="500" height="300">

### 2) LASSO(Least Absolute Shrinkage and Selection Operator)
* 라쏘는 최소 제곱 비용함수에 가중치의 크기를 추가한 L1 규제 모델로 희소한 모델을 만들 수 있다
* m>n이면 최대 n개의 특성을 선택하는 것이 한계
* ```lasso = Lasso(alpha=1.0)```
<img src="./img/lasso.PNG" width="500" height="300">

### 3)ElasticNet
* 릿지 회귀와 라쏘의 절충안
* ```elanet = ElasticNet(alpha=1.0, l1_ratio=0.5)```
* l1_ratio=1 이면 LASSO와 동일하지만, l1_ratio=0 이면 Ridge와 동일하지 않음
* 왜냐면, 𝜆1=alpha * l1_ratio & 𝜆2=alpha * (1-l1_ratio)/2 이기 때문
<img src="./img/elastic.PNG" width="500" height="300">


## [2]  Ensemble 
* 여러 개의 약한 분류기를 생성하고 학습시킨 뒤, 그 학습 결과를 결합으로써 과적합을 방지하고 보다 정확한 예측을 하는 기법

## [2-1]  Ensemble - Bagging(Bootstrap sample + Aggregating
* 앙상블에 있는 개별 분류기를 동일한 train data로 학습하는 것이 아니라 원본 train data에서 부트스트랩 샘플(중복을 허용한 랜덤 샘플)을 뽑아서 사용
* ex) Random Forest

In [10]:
m1=BaggingRegressor()
m1.fit(x_train, y_train)
p1=m1.predict(x_val)

print(f'RMES : {mean_squared_error(y_val, p1, squared=False)}')
print(f'MAE : {mean_absolute_error(y_val,p1)}')
print(f'MAPE : {mean_absolute_percentage_error(y_val,p1)}')

RMES : 5.321741095682285
MAE : 2.8742763157894737
MAPE : 0.14412295997533547


## [2-2]  Ensemble - Boosting
* 여러 개의 모델을 순차적으로 학습 및 예측하면서 잘못 예측한 데이터에 가중치를 부여해 오류를 개선해나가는 학습 방식
* 중복을 허용하지 않고 train data에서 랜덤 샘플을 추출하여 부분집합을 구성
* 유명한 AdaBoosts는 약한 학습기를 훈련할 때 훈련 세트를 전체 사용

In [11]:
m2_1=AdaBoostRegressor()
m2_1.fit(x_train, y_train)
p2_1=m2_1.predict(x_val)

print(f'RMES : {mean_squared_error(y_val, p2_1, squared=False)}')
print(f'MAE : {mean_absolute_error(y_val,p2_1)}')
print(f'MAPE : {mean_absolute_percentage_error(y_val,p2_1)}')

RMES : 4.993294268106011
MAE : 2.976678191328267
MAPE : 0.14554974450956065


In [12]:
m2_2=GradientBoostingRegressor()
m2_2.fit(x_train, y_train)
p2_2=m2_2.predict(x_val)

print(f'RMES : {mean_squared_error(y_val, p2_2, squared=False)}')
print(f'MAE : {mean_absolute_error(y_val,p2_2)}')
print(f'MAPE : {mean_absolute_percentage_error(y_val,p2_2)}')

RMES : 4.306161883606683
MAE : 2.496650588967645
MAPE : 0.12496382728019398


## [2-3] Ensemble - Stacking
* 개별 분류기가 예측한 데이터를 다시 train data로 새로운 분류기가 다시 예측을 수행하는 기법

In [15]:
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR

estimators=[('ridge', Ridge()), ('lasso', Lasso())]
m3=StackingRegressor(estimators=estimators, final_estimator=RandomForestRegressor())
m3.fit(x_train, y_train)
p3=m3.predict(x_val)

print(f'RMES : {mean_squared_error(y_val, p3, squared=False)}')
print(f'MAE : {mean_absolute_error(y_val,p3)}')
print(f'MAPE : {mean_absolute_percentage_error(y_val,p3)}')

RMES : 5.456951780696081
MAE : 3.294065789473685
MAPE : 0.15620802989695418


## [2-4] Ensemble - Voting
* 여러 모델을 통해 얻은 예측 결과들로부터 다수결 투표를 하여 최종 결과를 예측
* 하드 보팅 & 소프트 보팅
    * 하드 보팅 : 예측 결과 중 다수 선택된 값을 최종 결과로 선정
    * 소프트 보팅 : 각 분류기별 결정 확률을 평균내어 얻은 확률이 가장 높은 값을 최종 결과로 선정

In [17]:
model1=LinearRegression()
model2=RandomForestRegressor()
model3=KNeighborsRegressor()

m4=VotingRegressor([('lr',model1),('rf',model2), ('knn',model3)])
m4.fit(x_train, y_train)
p4=m4.predict(x_val)

print(f'RMES : {mean_squared_error(y_val, p4, squared=False)}')
print(f'MAE : {mean_absolute_error(y_val,p4)}')
print(f'MAPE : {mean_absolute_percentage_error(y_val,p4)}')

RMES : 5.003572295409232
MAE : 3.131906070227881
MAPE : 0.14186888480421894


# 3. 다양한 분류모델
* 회귀모델과 비슷 :)

# 4. AutoML(Automated Machine Learning)

* 머신러닝 모델 개발을 자동화하는 분야로, 모델 개발 과정에서 필요한 데이터 분할, 정제, feature 선택 및 추출, 모델 선택, 하이퍼파라미터 튜닝 등을 함수화하여 자동화 하는 것이다.
* AutoML의 핵심은 데이터가 주어졌을 때 가장 적절한 **모델**과 **파라미터**를 선정하는 것이다.
* 대표적인 오픈소스 AutomML tool & paper에는 Auto-WEAK, Hyperopt-Sklearn, Auto-Sklearn, Auto-Net(Auto-Pytorch), TPOT, Automatic Statistician, Auto-Keras, H20 Automl 등이 있다.
    * 이 중 Auto-WEKA에 대해 간략히 알아보면, **Sequential Model-Based Optimization(SMBO)**와 **Sequential Model-based Algorithm Configuration(SMAC)**를 통해 CASH(Combined Algorithm Selection and Hyperparameter) 문제를 해결하는 것이다.
    <img src="./img/cash.PNG" width="500" height="300">
    * 참고 : [Automated Machine Learning: Methods, Systems, Challenges]
* 추가적인 상용 도구로 Microsoft NNI, Google Automl 등이 있다. 

# 4-1. Pycaret
* AutoML 파이썬 라이브러리로, sklearn 패키지를 기반으로 분류, 회귀, 클러스터링, 이상치 탐지 등 다양한 모델 지원

In [1]:
# pip install pycaret

In [7]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

from sklearn.metrics import * 

from pycaret.regression import *

boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['target']=boston.target

train=df.iloc[:400,]
test=df.iloc[400:,]

# setup
s = setup(train, target='target')

,Description,Value
0,session_id,2351
1,Target,target
2,Original Data,"(400, 14)"
3,Missing Values,False
4,Numeric Features,12
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(279, 13)"


In [8]:
# 모델 선정 및 트레이닝
best5 = compare_models(sort='RMSE', n_select=5)

# # 최적 모델 분석
# evaluate_model(best5)

# # test data로 예측
# predictions = predict_model(best5, data=test)

# predictions

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,2.0430,8.1173,2.7701,0.8848,0.1255,0.0979,0.0290
catboost,CatBoost Regressor,2.0077,8.2372,2.7940,0.8893,0.1200,0.0924,0.9670
et,Extra Trees Regressor,2.1463,9.1252,2.9391,0.8744,0.1221,0.0974,0.0770
xgboost,Extreme Gradient Boosting,2.1886,9.3584,2.9954,0.8632,0.1303,0.1022,0.0510
rf,Random Forest Regressor,2.2118,10.6212,3.1106,0.8498,0.1325,0.1033,0.1160
ada,AdaBoost Regressor,2.4083,10.4508,3.1750,0.8496,0.1469,0.1183,0.0330
lightgbm,Light Gradient Boosting Machine,2.4285,12.9223,3.4712,0.8355,0.1461,0.1115,0.0130
dt,Decision Tree Regressor,2.8474,17.1699,3.8975,0.7672,0.1700,0.1370,0.0100
lar,Least Angle Regression,3.4018,23.9960,4.8036,0.6786,0.2494,0.1623,0.0100
lr,Linear Regression,3.4018,23.9960,4.8036,0.6786,0.2494,0.1623,0.0090


In [9]:
best5

[GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                           init=None, learning_rate=0.1, loss='ls', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=2351, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0, warm_start=False),
 ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     max_samples=None, min_impurity_decrease=0.0,
                     min_impurity_split=None, min_samples_leaf=1,
                     min_samples_split=2, m

In [11]:
# 모델 튜닝
tuned_best5 = [tune_model(i) for i in best5]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.3637,12.6431,3.5557,0.8466,0.1752,0.1286
1,1.9914,8.8450,2.9740,0.9053,0.0937,0.0745
2,2.8081,30.0562,5.4824,0.7188,0.1857,0.1232
3,2.9976,17.7855,4.2173,0.8233,0.1295,0.1058
4,2.2376,8.4781,2.9117,0.8943,0.1367,0.1131
5,2.0372,5.7195,2.3915,0.9199,0.1585,0.1321
6,2.7172,16.6471,4.0801,0.7737,0.1504,0.1234
7,3.4026,28.5284,5.3412,0.6433,0.1711,0.1351
8,2.5808,12.8005,3.5778,0.7504,0.1389,0.1034


In [12]:
tuned_best5

[GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                           init=None, learning_rate=0.2, loss='ls', max_depth=3,
                           max_features='log2', max_leaf_nodes=None,
                           min_impurity_decrease=0, min_impurity_split=None,
                           min_samples_leaf=3, min_samples_split=7,
                           min_weight_fraction_leaf=0.0, n_estimators=270,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=2351, subsample=0.9, tol=0.0001,
                           validation_fraction=0.1, verbose=0, warm_start=False),
 ExtraTreesRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse', max_depth=8,
                     max_features='sqrt', max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0002, min_impurity_split=None,
                     min_samples_leaf=2, min_samples_split=9,
                     mi

In [13]:
# 위 5개의 모델을 혼합만 모델 생성
blender_best5 = blend_models(estimator_list = tuned_best5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.7925,6.8967,2.6262,0.9163,0.1586,0.1058
1,2.0703,8.2293,2.8687,0.9119,0.0879,0.0741
2,2.2831,15.4640,3.9324,0.8553,0.1286,0.0988
3,2.2167,9.4750,3.0781,0.9058,0.1074,0.0825
4,1.8765,6.3001,2.5100,0.9215,0.1294,0.1007
5,1.4906,3.4289,1.8517,0.9520,0.1181,0.0934
6,2.5370,15.2752,3.9083,0.7923,0.1477,0.1174
7,2.8143,18.4948,4.3006,0.7688,0.1391,0.1118
8,2.0611,8.3748,2.8939,0.8367,0.1209,0.0865


In [14]:
blender_best5

VotingRegressor(estimators=[('gbr',
                             GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.2,
                                                       loss='ls', max_depth=3,
                                                       max_features='log2',
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=3,
                                                       min_samples_split=7,
                                                       min_weight_fraction_leaf=0.0,
                                   

In [15]:
# test data로 예측

final_model = finalize_model(blender_best5)
prediction = predict_model(final_model, data=test)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,3.0087,14.0386,3.7468,0.5032,0.2448,0.2389


In [16]:
prediction

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target,Label
400,25.04610,0.0,18.10,0.0,0.693,5.987,100.0,1.5888,24.0,666.0,20.2,396.90,26.77,5.6,9.573651
401,14.23620,0.0,18.10,0.0,0.693,6.343,100.0,1.5741,24.0,666.0,20.2,396.90,20.32,7.2,10.767080
402,9.59571,0.0,18.10,0.0,0.693,6.404,100.0,1.6390,24.0,666.0,20.2,376.11,20.31,12.1,12.105000
403,24.80170,0.0,18.10,0.0,0.693,5.349,96.0,1.7028,24.0,666.0,20.2,396.90,19.77,8.3,9.918435
404,41.52920,0.0,18.10,0.0,0.693,5.531,85.4,1.6074,24.0,666.0,20.2,329.46,27.38,8.5,9.654686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4,24.256774
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6,21.694695
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9,25.212875
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0,26.038082


In [17]:
# 예측 결과 평가
from pycaret.utils import check_metric
check_metric(prediction['target'], prediction['Label'], metric = 'RMSE')

3.7468

# 5. Bayesian Optimaization
#### [참고]https://sualab.github.io/introduction/practice/2019/02/19/bayesian-optimization-overview-1.html

* AutoML은 'ML을 설계하는 ML'로 현재 3가지 방향으로 연구가 진행됨.
    * 1. Automated Feature Learning
    * 2. Architecture Search
    * 3. Hyperparameter Optimization
* 이 중 Hyperparameter Optimization을 위한 주요 방법론과 함께, 학습 관점에서의 최적의 hyperparameter를 탐색하기 위한 방법인 Bayesian Optimization에 대해 알아봄.

## [1] Manual Search
* 가장 단순하고 직관적인 방법으로, 매 회차에 시도할 후보 hyperparameter 값을 주관적으로 선정하고 이를 사용하여 학습을 수행한 후 검증 데이터셋에 대하여 측정한 성능 결과를 기록하는 과정을 반복
* 이는 최적의 hyperparameter를 찾는 과정에서 은연 중에 발생하는 실험자의 편견으로 인해 실제 최적 값을 찾기가 상대적으로 어렵다는 단점이 있음

## [2] Grid Search & Random Serach
* Manual Search의 단점을 보완한 상대적으로 체계적인 방식
* Grid Search는 탐색의 대상이 되는 특정 구간 내의 후보 hyperparameter 값들을 일정한 간격을 두고 선정하여, 이들 각각에 대하여 측정한 성능 결과를 기록한 뒤 가장 높은 성능을 발휘했던 hyperparameter를 선정하는 방법
* Random Search는 Grid Search와 비슷하나 hyperparameter 값들을 랜덤 샘플링 한다는 특징으로, Grid Search에 비해 불필요한 반복 수행 횟수를 대폭 줄이면서 동시에 정해진 간격 사이에 위치한 값들에 대해서도 확률적으로 탐색이 가능하기에 더 빨리 찾을 수 있는 방법
* 위 두 방법 모두 바로 다음 번 시도할 후보 hyperparameter 값을 선정하는 과정에서 이전까지의 조사 과정에서 얻어진 값들의 성능 결과에 대한 '사전 지식'을 전혀 반영하지 못한다는 단점이 있음

## [3] Bayesian Optimization

### 1) 목표 및 가정
* Bayesian Optimization은 본래 어느 입력값 x를 받는 미지의 목적함수(objective function) f를 상정하여, 그 함수값 f(x)를 최대로 만든느 최적해 x*을 찾는 것을 목적으로 함
* 보통 목적함수의 표현식을 명시적으로 알지 못하며(blackbox function), 하나의 함수값 f(x)를 계싼하는데 오랜 시간이 소요되는 경우를 가정함
* 이러한 상황에서 가능한 적은 수의 입력값 후보들에 대해서만 그 함수값을 순차적으로 조사하여, f(x)를 최대로 만드는 최적해 x*를 빠르고 효과적으로 찾는 것이 주요 목표임

### 2) 필수요소
* Surrogate Model : 현재까지 조사된 (입력값, 함수값)들 (x_1, f(x_1)), ... , (x_t, f(x_t))를 바탕으로, 미지의 목적함수의 형태에 대한 확률적인 추정을 수행하는 모델
* Acquistion Function :  목적함수에 대한 현재까지의 확률적 추정결과를 바탕으로, '최적 입력값 x*를 찾는 데 있어 가장 유용할 만한' 다음 입력값 후보 x_t+1을 추천해주는 함수

### 3) Algorithm(pseudo code)
> * for t = 1, 2, ... do   
> * 1. 기존 (입력값, 함수값)들의 모음에 대한 Surrogate Model의 확률적 추정 결과를 바탕으로, Acquistion Function을 최대화하는 다음 입력값 후보 x_t+1을 선정
> * 2. 입력값 후보 x_t+1에 대한 함수값 f(x_t+1)을 계산
> * 3. 기존 (입력값 함수값)들의 모음에 (x_t+1, f(x_t+1))을 추가하고, Surrogate Model로 확률적 추정을 다시 수행
> * end for

### 4) Surrogate Model
* Surrogate Model로 가장 많이 사용되는 확률 모델은 Gaussian Process(GP)이다.
* 그 외에, 현재까지 조사된 (입력값, 함수값)들을 바탕으로 목적함수 추정에 있어서의 '불확실성'을 커버할 수 있는 모델은 Surrogate Model로 활용이 가능하며, Tree-structured Parzen Estimators(TPE), Deep Neural Networks 등이 있다.

#### Gausian Processes(GP)
* GP는 어느 특정 변수에 대한 확률 분포를 표현하는 보통의 확률 모델과는 다르게, 모종의 함수들에 대한 확률 분포를 나타내기 위한 확률 모델
* 그 구성요소들 간의 결합 분포(joint distribution)가 Gaussian distribution을 따른다는 특징
* GP는 평균함수(μ)와 공분산함수(k)를 사용하여 함수들에 대한 확률 분포로 표현
<img src="./img/GP.PNG" width="200" height="150">
* 과정 
    * 검은색 점선 : 실제 목적 함수
    * 검은색 실선 : 추정된 평균 함수
    * 검은색 점   : 현재까지 조사된 (입력값, 함수값)
    * 파란색 음영 : 추정된 표준편차
    * 녹색 실선   : Acquistion Function
    * 조사된 점으로부터 거리가 먼 x일수록, 그 지점에 대해 추정한 평균값의 '불확실성'이 크다는 의미를 내포
<img src="./img/Bayesian Optimization.PNG" width="400" height="300">

[출처](chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://arxiv.org/pdf/1012.2599.pdf) A Tutorial on Bayesian Optimization of Expensive Cost Functions, with Application to Active User Modeling and Hierarchical Reinforcement Learning 

### 5) Acquisiton Function
* Exploitation(착취, 수탈)
    * t=2인 상황에서 조사된 (x, f(x)) 점들은 2개인데, 지금 상황에서 오로지 이들만 놓고 생각해보면, 두 점 중에서 '함수값이 더 큰 점 근방에서 실제 최적 입력값 x*를 찾을 가능성이 높을 것이다'라고 예측하는 것은 어느 정도 그럴싸한 예측이다
* Exploration(탐색)
    * 관점을 달리하여, 현재까지 조사된 두 점 사이에 위치했으면서 표준편차가 큰 영역의 경우, 이 부분의 추정된 평균 함수값이 실제 목적 함수값과 유사할 것이라고 장담하기 매우 어려울 것임을 직관적으로 알 수 있다. 그렇지만, '불확신한 영역에 최적 입력값 x*가 존재할 가능성이 있으므로 이 부분을 추가적으로 탐색해야 한다'고 생각하는 것이 어느 정도 그럴싸한 판단이라고 할 수 있으며, 이에 따라 현재까지 추정된 목적함수 상에서 표준편차가 최대인 점 근방을 그 다음 차례에 시도하는 것 또한 나름대로 합리적인 전략이다.
* Exploitation 전략과 Exploration 전략 모두, 최적 x*를 효과적으로 찾는 데 있어 균등하게 중요한 접근 적략이라 할 수 있으나, 두 전략의 성격이 서로 trade-off 관계에 있다는 것이다. 그렇기에 두 전략간 상대적 강도를 적절하게 조절하는 것이 실제 목적함수에 대한 성공적인 최적 입력값 탐색에 매우 중요하다 할 수 있다.

#### Expected Improvement(EI)
* EI는 Exploitation 전략과 Exploration 전략 모두 내재적으로 일정 수준 포함하도록 설계된 것으로, Acquistion Function으로 가장 많이 사용된다.
* 현재까지 추정된 목점함수를 바탕으로 어느 후보 입력값 x에 대하여 '현재까지 조사된 점들의 함수값 중 최대 함수값보다 더 큰 함수값을 도출할 확률(PI)' 및 '그 함수값과 최대 함수값 간의 차이값'을 종합적으로 고려하여, 해당 입력값 x의 유용성을 나타내는 숫자를 출력한다.   

<img src="./img/EI.PNG" width="400" height="500">

* 그 외에 '현재까지 조사된 점들의 함수값 주 최대 함수값보다 더 큰 함수값을 도출할 확률'만을 반영한 Probability of Improvement(PI)와, Upper Confidence Bound(UUCB), Entropy Search(ES) 등이 Acquisition Function으로 사용된다.
    * GP 사용시, 최대 함수값보다 더 큰 함수값을 도출할 확률(PI)에 대한 시각화 예시
        * 세로 방향 점선 : 후보 입력값 x1,x2,x3에서의 함수값 f(x1),f(x2),f(x3) 각각에 대한 확률 분포
        * 초록색 음영    : f(x3)의 확률 분포상에서 그 값이 최대 함수값보다 큰 영역   
        
<img src="./img/PI.PNG" width="500" height="500">

[출처](chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://arxiv.org/pdf/1012.2599.pdf) A Tutorial on Bayesian Optimization of Expensive Cost Functions, with Application to Active User Modeling and Hierarchical Reinforcement Learning 

## [4] 결론
* 딥러닝에서 Hyperparameter Optimization이란, 딥러닝 모델의 학습을 수행하기 위해 사전에 설정해야하는 값인 Hyperparameter의 최적값을 탐색하는 문제를 지칭한다.
* Bayesian Optimization은 매 회 새로운 hyperparameter 값에 대한 조사를 수행할 시 '사전 지식'을 충분히 반영하면서, 동시에 전체적인 탐색 과정을 좀 더 체계적으로 수행하기 위해 고려해볼 수 있는 Hyperparameter Optimization 방법론이다.
* 첫번째 구성요소인 Surrogate Model은 현재까지 조사된 (입력값, 함수값) 점들을 바탕으로 어느 미지의 목적함수에 대한 확률적인 추정을 수행하는 모델을 지칭하며, Gaussian Process가 대표적이다.
* 두번째 구성요소인 Acquisiton Function은 목적함수에 대한 현재까지의 확률적 추정 결과를 바탕으로 '최적 입력값을 찾는데 있어 가장 유용할 만한' 다음 입력값 후보를 추전해주는 함수를 지칭하며, Expected Improvement가 대표적이다.

# +. ETC

## [1] 모델 저장 

!pip install joblib   
joblib.dump( 'model' , '경로')

## [2] Pandas Profiling - EDA

In [1]:
# pip install pandas-profiling
# pip install markupsafe==2.0.1

In [5]:
from pandas_profiling import ProfileReport

profile=ProfileReport(df, title='boston Report')

profile.to_file('boston Report.html')

Summarize dataset:   0%|          | 0/28 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\User\anaconda3\lib\site-packages\pandas_profiling\visualisation\plot.py:154: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("RdBu"))
  cmap.set_bad(cmap_bad)


Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]